In [1]:
import numpy as np
import pandas as pd

#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, ETS

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut
from hierarchicalforecast.methods import MinTrace
from hierarchicalforecast.utils import aggregate

import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('C:\\Users\\IqraImtiaz\\OneDrive - keystonestrategy.com\\Documents\\Moderna\\IQVIA\\iqvia_zip.csv')
df['Date'] = pd.to_datetime(df['Date'])

data = df
data = data.dropna()
data = data.dropna(subset=['PROVIDER_ZIP'])
data["PROVIDER_ZIP"] = data["PROVIDER_ZIP"].astype(str).str.strip()
data = data[data['PROVIDER_ZIP'] != "\"\""]
data["PROVIDER_ZIP"] = data["PROVIDER_ZIP"].str.replace("\"","")
data = data.sort_values(by=['PROVIDER_ZIP', 'Date'])

data['zip'] = data['PROVIDER_ZIP'].astype(str).str.split('.',expand=True)[0]
data['zip'] = np.where(data['zip'].str.len()==3,'00'+data['zip'],np.where(data['zip'].str.len()==4,'0'+data['zip'],data['zip']))
data['zip3'] = data['zip'].str.slice(stop=3)
data = data[['Date','zip3','CLM_CNT']]
data = data.groupby(['zip3','Date']).sum()

# filter on xip codes that have at least 10 data points
data = data.groupby('zip3').filter(lambda x: len(x)>10)
data.reset_index(inplace=True)

grouped_sum = data.groupby('Date')['CLM_CNT'].transform('sum')
data['natl_admins'] = grouped_sum
data['shares'] = (data['CLM_CNT'] / data['natl_admins'])
df = data
df

,zip3,Date,CLM_CNT,natl_admins,shares
0,006,2022-01-07,13087,4488095,0.002916
1,006,2022-01-14,21158,4523297,0.004678
2,006,2022-01-21,18672,3465159,0.005388
3,006,2022-01-28,15378,2847044,0.005401
4,006,2022-02-04,12245,2106287,0.005814
...,...,...,...,...,...
93444,999,2023-12-29,11,486757,0.000023
93445,999,2024-01-05,11,535153,0.000021
93446,999,2024-01-12,12,496408,0.000024
93447,999,2024-01-19,9,328782,0.000027


In [3]:
# Create a complete set of all possible combinations of zip3 and dates
all_dates = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='W-FRI')
all_zip3 = df['zip3'].unique()
complete_index = pd.MultiIndex.from_product([all_zip3, all_dates], names=['zip3', 'Date'])

# Reindex the DataFrame to include all combinations, filling missing values with zeros
df_complete = df.set_index(['zip3', 'Date']).reindex(complete_index, fill_value=0).reset_index()

# Prepare the data for forecasting
base_df = df_complete[['zip3', 'Date', 'CLM_CNT']].rename(columns={'zip3': 'zip3', 'Date': 'ds', 'CLM_CNT': 'y'})

In [4]:
spec = [
    ['zip3']
]

In [5]:

Y_df, S_df, tags = aggregate(base_df, spec)
Y_df = Y_df.reset_index()
Y_df.head()


,unique_id,ds,y
0,006,2022-01-07,13087
1,006,2022-01-14,21158
2,006,2022-01-21,18672
3,006,2022-01-28,15378
4,006,2022-02-04,12245


In [6]:
Y_test_df = Y_df.groupby('unique_id').tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

Y_train_df.groupby('unique_id').size()

unique_id
006    100
007    100
008    100
009    100
010    100
      ... 
995    100
996    100
997    100
998    100
999    100
Length: 888, dtype: int64

In [7]:
fcst = StatsForecast(df=Y_train_df, 
                     models=[ETS(season_length=7, model='ZZA')], 
                     freq='W', n_jobs=-1)
Y_hat_df = fcst.forecast(h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()


c:\Users\IqraImtiaz\.conda\envs\nixtla\lib\site-packages\statsforecast\models.py:880: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()
c:\Users\IqraImtiaz\.conda\envs\nixtla\lib\site-packages\statsforecast\core.py:474: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
c:\Users\IqraImtiaz\.conda\envs\nixtla\lib\site-packages\statsforecast\core.py:626: FutureWarning: Passing unique_id as the index is deprecated. Please provide it as a column instead.
  warnings.warn(
c:\Users\IqraImtiaz\.conda\envs\nixtla\lib\site-packages\statsforecast\core.py:474: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will rais

In [8]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)


In [9]:
def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

eval_tags = {}
eval_tags['zip3'] = tags['zip3']
eval_tags['All'] = np.concatenate(list(tags.values()))

evaluator = HierarchicalEvaluation(evaluators=[rmse, mase])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
evaluation = evaluation.drop('Overall')
evaluation.columns = ['Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
evaluation = evaluation.applymap('{:.2f}'.format)


C:\Users\IqraImtiaz\AppData\Local\Temp\ipykernel_14572\708716589.py:18: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  evaluation = evaluation.drop('Overall')
C:\Users\IqraImtiaz\AppData\Local\Temp\ipykernel_14572\708716589.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  evaluation = evaluation.applymap('{:.2f}'.format)


In [10]:
evaluation

Base BottomUp MinTrace(mint_shrink) MinTrace(ols)
level metric                                                     
zip3  rmse    499.41   499.41                499.41        499.41
      mase      0.64     0.64                  0.64          0.64
All   rmse    499.41   499.41                499.41        499.41
      mase      0.64     0.64                  0.64          0.64

In [11]:
Y_test_df

,ds,y
unique_id,,
006,2023-12-08,746
006,2023-12-15,475
006,2023-12-22,488
006,2023-12-29,559
006,2024-01-05,382
...,...,...
999,2023-12-29,11
999,2024-01-05,11
999,2024-01-12,12


In [12]:
Y_rec_df

,ds,ETS,ETS/BottomUp,ETS/MinTrace_method-mint_shrink,ETS/MinTrace_method-ols
unique_id,,,,,
006,2023-12-03,921.366272,921.366272,921.366272,921.366272
006,2023-12-10,880.782471,880.782471,880.782471,880.782471
006,2023-12-17,790.062439,790.062439,790.062439,790.062439
006,2023-12-24,778.720764,778.720764,778.720764,778.720764
006,2023-12-31,1082.200684,1082.200684,1082.200684,1082.200684
...,...,...,...,...,...
999,2023-12-24,-19.636202,-19.636202,-19.636202,-19.636202
999,2023-12-31,-4.757237,-4.757237,-4.757237,-4.757237
999,2024-01-07,16.876272,16.876272,16.876272,16.876272


In [13]:
import matplotlib.pyplot as plt

# Reset the index to make sure we can merge on 'unique_id' and 'ds'
Y_test_df = Y_test_df.reset_index()
Y_rec_df = Y_rec_df.reset_index()

# Merge actuals and forecasts
comparison_df = pd.merge(Y_test_df, Y_rec_df, on=['unique_id', 'ds'])

# Plotting
for unique_id in comparison_df['unique_id'].unique():
    data = comparison_df[comparison_df['unique_id'] == unique_id]
    
    plt.figure(figsize=(10, 6))
    plt.plot(data['ds'], data['y'], label='Actual', marker='o')
    plt.plot(data['ds'], data['ETS'], label='Forecast - ETS', linestyle='--', marker='x')
    plt.plot(data['ds'], data['ETS/BottomUp'], label='Forecast - BottomUp', linestyle='--', marker='x')
    plt.plot(data['ds'], data['ETS/MinTrace_method-mint_shrink'], label='Forecast - MinTrace (mint_shrink)', linestyle='--', marker='x')
    plt.plot(data['ds'], data['ETS/MinTrace_method-ols'], label='Forecast - MinTrace (ols)', linestyle='--', marker='x')
    plt.title(f"Actual vs Forecast for {unique_id}")
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()


In [14]:
Y_test_df

,unique_id,ds,y
0,006,2023-12-08,746
1,006,2023-12-15,475
2,006,2023-12-22,488
3,006,2023-12-29,559
4,006,2024-01-05,382
...,...,...,...
7099,999,2023-12-29,11
7100,999,2024-01-05,11
7101,999,2024-01-12,12
7102,999,2024-01-19,9
